In [1]:
import random
import numpy as np
import stimrgs_v1 as pan_lib
import stim

In [ ]:
def generate_rgs_random(num_nodes: int, num_bell_between_row: int) -> tuple[stim.TableauSimulator, str]:
    """Generate a random RGS circuit with the given number of nodes and bells.
    
    Args:
        num_nodes (int): The number of nodes in the RGS system (3arms -> 12 nodes, 4arms -> 16 nodes).
        num_bell_between_row (int): The number of bells want to randomly generate between row1 and row2, as well as, row3 and row4.
    
    Returns:
        stim.TableauSimulator, str
        rgs's queries from input configuration, and the generated RGS circuit as a string.
    """
    edges = []
    queries = []

    if num_nodes % 4 != 0:
        raise ValueError("Number of nodes should be a multiple of 4.")
    if num_bell_between_row > num_nodes/4:
        raise ValueError("Number of Bell pairs should be less than or equal to the number of nodes in each row.")
    
    num_nodes_each_row = int(num_nodes/4)
    row_1 = range(0, num_nodes_each_row)
    row_2 = range(num_nodes_each_row, 2*num_nodes_each_row)
    row_3 = range(2*num_nodes_each_row, 3*num_nodes_each_row)
    row_4 = range(3*num_nodes_each_row, 4*num_nodes_each_row)

    for i in range(num_nodes):
        q = ['_'] * num_nodes 
        q[i] = 'X'
        queries.append(q)

    queries_result = []
    for index, q in enumerate(queries):    

        if index in row_1:
            random_top = random.sample(row_2, num_bell_between_row)              
            for idx in random_top:
                q[idx] = 'Z'    
                queries[idx][index] = 'Z'
                edges.extend([index, idx])
        
        if index in row_2:
            # index = 2,3   z at 4,5
            # next to add z at 2,3 when index = 4,5
            q[index + num_nodes//4] = 'Z'        
                        
            edges.extend([index, index + num_nodes//4])

        if index in row_3:
            random_bottom = random.sample(row_4, num_bell_between_row)            
            for idx in random_bottom:
                q[idx] = 'Z'        
                queries[idx][index] = 'Z'  
                edges.extend([index, idx])

                q[index - num_nodes//4] = 'Z'

        _q = ''.join(q)
        queries_result.append(_q)
    
    circuit = pan_lib.utils.generate_h_cz_string(edges, fix_nodes=True, num_nodes=num_nodes)

    return queries_result, circuit 

In [3]:
generate_rgs_random(12, 2)

(['X__ZZ_______',
  '_X_ZZ_______',
  '__XZZ_______',
  'ZZZX__Z_____',
  'ZZZ_X__Z____',
  '_____X__Z___',
  '___Z__X__Z_Z',
  '____Z__X_ZZ_',
  '_____Z__XZ_Z',
  '______ZZZX__',
  '_______Z__X_',
  '______Z_Z__X'],
 'H 0 1 2 3 4 5 6 7 8 9 10 11\nCZ 0 3 0 4 1 3 1 4 2 4 2 3 3 6 4 7 5 8 6 9 6 11 7 9 7 10 8 11 8 9')